In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

import cifar10

cifar10.maybe_download_and_extract()

images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

#extract mean
mean = np.mean(images_train,axis=(0,1,2,3))
images_train=images_train-mean
images_test=images_test-mean

X_val=images_train[:500,:,:,:]
y_onehot_val=labels_train[:500,:]
X_train=images_train[500:,:,:,:]
y_onehot=labels_train[500:,:]

Data has apparently already been downloaded and unpacked.
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [8]:
# evaluate performance on some data 
def perf_eval(logit_pred, y_true):
    """a function to evaluate performance of predicted y values vs true class labels"""
    # now look at some data
    print('    sample pred: {0}\n    sample true: {1}'.format(np.argmax(logit_pred[0:20],1),np.argmax(y_true[0:20],1)))
    # avg accuracy
    is_correct_vals = np.equal(np.argmax(logit_pred,1),np.argmax(y_true,1))
    #accuracy_vals = np.mean(is_correct_vals)
    #print('    mean classification accuracy: {0}%'.format(100*accuracy_vals))
    # Dig in a little deeper.  Where did we make correct predictions?  Does this seem reasonable?
    print('    correct predictions by class: {0}'.format(y_true[is_correct_vals,:].sum(axis=0)))
    
# cnn conv stuff
def conv(x, W):
    """simple wrapper for tf.nn.conv2d"""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool(x):
    """simple wrapper for tf.nn.max_pool with stride size 2"""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def compute_cross_entropy(logits, y):
    # Compute the average cross-entropy across all the examples.
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') # the predicted probability for each example.
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        print(logits,y)
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy

def test_model(opt_method,model_type,batch_size,iterations):
    start_time = time.time()
    dir_name = 'logs/scratch04x/'
    n=50000
    n_val=500
    
    with tf.Graph().as_default():
    # We build the model here as before
        x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
        y = tf.placeholder(tf.float32, [None, 10], name='y')
        pkeep = tf.placeholder(tf.float32, name='pkeep')

        with tf.name_scope('model'):
            logits,weights = compute_logits(x, model_type, pkeep)
        with tf.name_scope('loss'):
            loss = compute_cross_entropy(logits=logits, y=y)
            regularizers=tf.nn.l2_loss(weights[0])
            for i in range(1,len(weights)):
                regularizers += tf.nn.l2_loss(weights[i])
            loss = tf.reduce_mean(loss + 0.0005 * regularizers)
        with tf.name_scope('accuracy'):
            accuracy = compute_accuracy(logits, y)

        with tf.name_scope('opt'):
            if opt_method == 'sgd':
                opt = tf.train.GradientDescentOptimizer(0.5)
            elif opt_method == 'rms':
                opt = tf.train.RMSPropOptimizer(.001)
            elif opt_method == 'adam':
                opt = tf.train.AdamOptimizer(1e-4)
            train_step = opt.minimize(loss)

        with tf.Session() as sess:
            #summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
            #summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
            #summary_writer_val = tf.summary.FileWriter(dir_name+'/val')

            sess.run(tf.global_variables_initializer())
            
            train_ac=[]
            val_ac=[]
            
            for i in range(iterations):
                batch = np.floor(np.random.rand(batch_size)*(n-n_val)).astype(int)
                X_batch = X_train[batch,:,:,:].reshape([batch_size,-1])
                y_batch = y_onehot[batch]

                # now run
                _  = sess.run((train_step),feed_dict={x: X_batch, y: y_batch, pkeep:0.85})

                # write the summary output to file
                #if i%100==0:
                    #summary_writer_train.add_summary(summary, i)

                # print diagnostics
                if i%50 == 0:
                    X_batch = X_train[0:500,:,:,:].reshape([500,-1])
                    y_batch = y_onehot[0:500]
                    (train_error,train_logits) = sess.run((accuracy,logits), {x: X_batch, y: y_batch, pkeep:1.0})
                    print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_error), flush=True)
                    # further diagnostics
                    perf_eval(train_logits, y_batch)

                if i%50 == 0:
                    X_batch = X_val.reshape([n_val,-1])
                    y_batch = y_onehot_val
                    (val_error) = sess.run((accuracy), {x:X_batch, y:y_batch, pkeep:1.0})
                    print("\rStep {0:3d}: val accuracy {1:0.4f}".format(i, val_error), flush=True)
                    
                    train_ac.append(train_error)
                    val_ac.append(val_error)
                    #summary_writer_val.add_summary(summary, i)
    
    end_time = time.time()
    time_dif = end_time - start_time
    print (val_ac,train_ac)
    print (time_dif/60)
    return val_ac,train_ac,time_dif/60

In [13]:
def compute_logits(x, model_type, pkeep):
    if model_type=='vgg3':
        x = tf.reshape(x, [-1,32,32,3])
        #cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[3, 3, 3, 32])
        b_conv1 = tf.get_variable('b_conv1', shape=[32])
        h_conv1 = tf.nn.relu(tf.add(conv(x, W_conv1), b_conv1))
        #pool 1
        pool1=maxpool(h_conv1)
        #cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[3, 3, 32, 64])
        b_conv2 = tf.get_variable('b_conv2', shape=[64])
        h_conv2 = tf.nn.relu(tf.add(conv(pool1, W_conv2), b_conv2))
        #pool 2
        pool2=maxpool(h_conv2)
        #cnn layer 3
        W_conv3 = tf.get_variable('W_conv3', shape=[3, 3, 64, 128])
        b_conv3 = tf.get_variable('b_conv3', shape=[128])
        h_conv3 = tf.nn.relu(tf.add(conv(pool2, W_conv3), b_conv3))
        #pool 3
        pool3=maxpool(h_conv3)
        # fc layer to logits
        shape=int(np.prod(pool3.get_shape()[1:]))
        flat = tf.reshape(pool3, [-1, shape])
        W_fc1 = tf.get_variable('W_fc1', [shape, 10])
        b_fc1 = tf.get_variable('b_fc1', [10])
        logits = tf.add(tf.matmul(flat, W_fc1), b_fc1, name='logits_vgg4_small_filters')
        weights=[W_conv1,W_conv2]
    elif model_type=='vgg14':
        x = tf.reshape(x, [-1,32,32,3])
        #cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[3, 3, 3, 64])
        b_conv1 = tf.get_variable('b_conv1', shape=[64])
        h_conv1 = tf.nn.relu(tf.add(conv(x, W_conv1), b_conv1))
        #cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[3, 3, 64, 64])
        b_conv2 = tf.get_variable('b_conv2', shape=[64])
        h_conv2 = tf.nn.relu(tf.add(conv(h_conv1, W_conv2), b_conv2))
        #pool 1
        pool1=maxpool(h_conv2)
        #cnn layer 3
        W_conv3 = tf.get_variable('W_conv3', shape=[3, 3, 64, 128])
        b_conv3 = tf.get_variable('b_conv3', shape=[128])
        h_conv3 = tf.nn.relu(tf.add(conv(pool1, W_conv3), b_conv3))
        #cnn layer 4
        W_conv4 = tf.get_variable('W_conv4', shape=[3, 3, 128, 128])
        b_conv4 = tf.get_variable('b_conv4', shape=[128])
        h_conv4 = tf.nn.relu(tf.add(conv(h_conv3, W_conv4), b_conv4))
        #pool 2
        pool2=maxpool(h_conv4)
        #cnn layer 5
        W_conv5 = tf.get_variable('W_conv5', shape=[3, 3, 128, 256])
        b_conv5 = tf.get_variable('b_conv5', shape=[256])
        h_conv5 = tf.nn.relu(tf.add(conv(pool2, W_conv5), b_conv5))
        #cnn layer 6
        W_conv6 = tf.get_variable('W_conv6', shape=[3, 3, 256, 256])
        b_conv6 = tf.get_variable('b_conv6', shape=[256])
        h_conv6 = tf.nn.relu(tf.add(conv(h_conv5, W_conv6), b_conv6))
        #cnn layer 7
        W_conv7 = tf.get_variable('W_conv7', shape=[3, 3, 256, 256])
        b_conv7 = tf.get_variable('b_conv7', shape=[256])
        h_conv7 = tf.nn.relu(tf.add(conv(h_conv6, W_conv7), b_conv7))
        #pool 3
        pool3=maxpool(h_conv7)
        #cnn layer 8
        W_conv8 = tf.get_variable('W_conv8', shape=[3, 3, 256, 512])
        b_conv8 = tf.get_variable('b_conv8', shape=[512])
        h_conv8 = tf.nn.relu(tf.add(conv(pool3, W_conv8), b_conv8))
        #cnn layer 9
        W_conv9 = tf.get_variable('W_conv9', shape=[3, 3, 512, 512])
        b_conv9 = tf.get_variable('b_conv9', shape=[512])
        h_conv9 = tf.nn.relu(tf.add(conv(h_conv8, W_conv9), b_conv9))
        #cnn layer 10
        W_conv10 = tf.get_variable('W_conv10', shape=[3, 3, 512, 512])
        b_conv10 = tf.get_variable('b_conv10', shape=[512])
        h_conv10 = tf.nn.relu(tf.add(conv(h_conv9, W_conv10), b_conv10))
        #pool 4
        pool4=maxpool(h_conv10)
        #cnn layer 11
        W_conv11 = tf.get_variable('W_conv11', shape=[3, 3, 512, 512])
        b_conv11 = tf.get_variable('b_conv11', shape=[512])
        h_conv11 = tf.nn.relu(tf.add(conv(pool4, W_conv11), b_conv11))
        #cnn layer 12
        W_conv12 = tf.get_variable('W_conv12', shape=[3, 3, 512, 512])
        b_conv12 = tf.get_variable('b_conv12', shape=[512])
        h_conv12 = tf.nn.relu(tf.add(conv(h_conv11, W_conv12), b_conv12))
        #cnn layer 13
        W_conv13 = tf.get_variable('W_conv13', shape=[3, 3, 512, 512])
        b_conv13 = tf.get_variable('b_conv13', shape=[512])
        h_conv13 = tf.nn.relu(tf.add(conv(h_conv12, W_conv13), b_conv13))
        #pool 5
        pool5=maxpool(h_conv13)
        #fc1-4096
        shape=int(np.prod(pool5.get_shape()[1:]))
        pool5_flat=tf.reshape(pool5,[-1,shape])
        W_fc1 = tf.get_variable('W_fc1', shape=[shape,4096])
        b_fc1 = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),name='b_fc1')
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(pool5_flat, W_fc1), b_fc1))
        # fc layer to logits
        W_fc = tf.get_variable('W_fc', [4096, 10])
        b_fc = tf.get_variable('b_fc', [10])
        logits = tf.add(tf.matmul(h_fc1, W_fc), b_fc, name='logits_vgg13')
        weights=[W_conv11,W_conv12,W_conv13,W_fc1]
        #,W_conv3,W_conv4,W_conv5,W_conv6,W_conv7,W_conv
    elif model_type=='vgg16':
        x = tf.reshape(x, [-1,32,32,3])
        #cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[3, 3, 3, 64])
        b_conv1 = tf.get_variable('b_conv1', shape=[64])
        h_conv1 = tf.nn.relu(tf.add(conv(x, W_conv1), b_conv1))
        #cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[3, 3, 64, 64])
        b_conv2 = tf.get_variable('b_conv2', shape=[64])
        h_conv2 = tf.nn.relu(tf.add(conv(h_conv1, W_conv2), b_conv2))
        #pool 1
        pool1=maxpool(h_conv2)
        #cnn layer 3
        W_conv3 = tf.get_variable('W_conv3', shape=[3, 3, 64, 128])
        b_conv3 = tf.get_variable('b_conv3', shape=[128])
        h_conv3 = tf.nn.relu(tf.add(conv(pool1, W_conv3), b_conv3))
        #cnn layer 4
        W_conv4 = tf.get_variable('W_conv4', shape=[3, 3, 128, 128])
        b_conv4 = tf.get_variable('b_conv4', shape=[128])
        h_conv4 = tf.nn.relu(tf.add(conv(h_conv3, W_conv4), b_conv4))
        #pool 2
        pool2=maxpool(h_conv4)
        #cnn layer 5
        W_conv5 = tf.get_variable('W_conv5', shape=[3, 3, 128, 256])
        b_conv5 = tf.get_variable('b_conv5', shape=[256])
        h_conv5 = tf.nn.relu(tf.add(conv(pool2, W_conv5), b_conv5))
        #cnn layer 6
        W_conv6 = tf.get_variable('W_conv6', shape=[3, 3, 256, 256])
        b_conv6 = tf.get_variable('b_conv6', shape=[256])
        h_conv6 = tf.nn.relu(tf.add(conv(h_conv5, W_conv6), b_conv6))
        #cnn layer 7
        W_conv7 = tf.get_variable('W_conv7', shape=[3, 3, 256, 256])
        b_conv7 = tf.get_variable('b_conv7', shape=[256])
        h_conv7 = tf.nn.relu(tf.add(conv(h_conv6, W_conv7), b_conv7))
        #pool 3
        pool3=maxpool(h_conv7)
        #cnn layer 8
        W_conv8 = tf.get_variable('W_conv8', shape=[3, 3, 256, 512])
        b_conv8 = tf.get_variable('b_conv8', shape=[512])
        h_conv8 = tf.nn.relu(tf.add(conv(pool3, W_conv8), b_conv8))
        #cnn layer 9
        W_conv9 = tf.get_variable('W_conv9', shape=[3, 3, 512, 512])
        b_conv9 = tf.get_variable('b_conv9', shape=[512])
        h_conv9 = tf.nn.relu(tf.add(conv(h_conv8, W_conv9), b_conv9))
        #cnn layer 10
        W_conv10 = tf.get_variable('W_conv10', shape=[3, 3, 512, 512])
        b_conv10 = tf.get_variable('b_conv10', shape=[512])
        h_conv10 = tf.nn.relu(tf.add(conv(h_conv9, W_conv10), b_conv10))
        #pool 4
        pool4=maxpool(h_conv10)
        #cnn layer 11
        W_conv11 = tf.get_variable('W_conv11', shape=[3, 3, 512, 512])
        b_conv11 = tf.get_variable('b_conv11', shape=[512])
        h_conv11 = tf.nn.relu(tf.add(conv(pool4, W_conv11), b_conv11))
        #cnn layer 12
        W_conv12 = tf.get_variable('W_conv12', shape=[3, 3, 512, 512])
        b_conv12 = tf.get_variable('b_conv12', shape=[512])
        h_conv12 = tf.nn.relu(tf.add(conv(h_conv11, W_conv12), b_conv12))
        #cnn layer 13
        W_conv13 = tf.get_variable('W_conv13', shape=[3, 3, 512, 512])
        b_conv13 = tf.get_variable('b_conv13', shape=[512])
        h_conv13 = tf.nn.relu(tf.add(conv(h_conv12, W_conv13), b_conv13))
        #pool 5
        pool5=maxpool(h_conv13)
        #fc1-4096
        shape=int(np.prod(pool5.get_shape()[1:]))
        pool5_flat=tf.reshape(pool5,[-1,shape])
        W_fc1 = tf.get_variable('W_fc1', shape=[shape,4096])
        b_fc1 = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),name='b_fc1')
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(pool5_flat, W_fc1), b_fc1))
        #fc2-4096
        W_fc2 = tf.get_variable('W_fc2', shape=[4096,4096])
        b_fc2 = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),name='b_fc2')
        h_fc2 = tf.nn.relu(tf.add(tf.matmul(h_fc1, W_fc2), b_fc2))
        #fc3-1000
        W_fc3 = tf.get_variable('W_fc3', shape=[4096,1000])
        b_fc3 = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),name='b_fc3')
        h_fc3 = tf.nn.relu(tf.add(tf.matmul(h_fc2, W_fc3), b_fc3))
        # fc layer to logits
        W_fc = tf.get_variable('W_fc', [1000, 10])
        b_fc = tf.get_variable('b_fc', [10])
        logits = tf.add(tf.matmul(h_fc3, W_fc), b_fc, name='logits_vgg13')
        weights=[W_conv6,W_conv8,W_conv10,W_conv11,W_conv12,W_conv13,W_fc1,W_fc2]
    return logits,weights

In [10]:
accuracy1_val,accuracy1_train,time1=test_model('adam','vgg3',64,301)

Tensor("model/logits_vgg4_small_filters:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.1080
    sample pred: [8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.   0.   0.   0.   0.  54.   0.]
Step   0: val accuracy 0.0900
Step  50: training accuracy 0.1740
    sample pred: [2 0 2 0 0 2 2 2 0 2 2 0 9 1 1 2 0 2 0 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 42.   6.  11.   3.   5.   0.   0.   7.   6.   7.]
Step  50: val accuracy 0.1780
Step 100: training accuracy 0.2640
    sample pred: [7 8 7 9 0 7 7 7 9 6 6 9 9 7 7 6 7 7 9 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 27.   3.   0.   1.   2.  11.  16.  30.  15.  27.]
Step 100: val accuracy 0.2520
Step 150: training accuracy 0.3060
    sample pred: [6 8 6 8 0 9 6 5 9 6 6 

In [15]:
accuracy1_val,accuracy1_train,time1=test_model('adam','vgg14',64,8001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.0900
    sample pred: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.   0.  45.   0.   0.   0.   0.]
Step   0: val accuracy 0.0800
Step  50: training accuracy 0.1120
    sample pred: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.  56.   0.   0.   0.   0.   0.   0.   0.]
Step  50: val accuracy 0.0860
Step 100: training accuracy 0.0980
    sample pred: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.  49.   0.   0.   0.   0.   0.]
Step 100: val accuracy 0.1000
Step 150: training accuracy 0.0900
    sample pred: [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3

Step 1500: training accuracy 0.4740
    sample pred: [3 1 5 8 2 9 9 9 1 1 9 9 0 4 6 6 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 24.  21.  14.  25.  17.  10.  35.  32.  30.  29.]
Step 1500: val accuracy 0.4920
Step 1550: training accuracy 0.4600
    sample pred: [5 8 7 8 0 9 1 8 1 1 7 9 0 7 7 6 7 7 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 34.  36.  13.   2.  11.  16.  28.  48.  31.  11.]
Step 1550: val accuracy 0.5000
Step 1600: training accuracy 0.4660
    sample pred: [3 1 5 8 0 9 9 8 1 1 9 9 9 4 4 6 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 20.  36.   8.  31.  28.   6.  29.  18.  35.  22.]
Step 1600: val accuracy 0.4940
Step 1650: training accuracy 0.4800
    sample pred: [5 8 5 8 0 9 1 8 1 8 9 9 0 4 4 6 7 4 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 3050: training accuracy 0.6460
    sample pred: [5 8 7 8 0 4 9 8 9 8 9 9 9 2 3 4 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 32.  32.  33.  27.  23.  18.  41.  34.  46.  37.]
Step 3050: val accuracy 0.6280
Step 3100: training accuracy 0.6260
    sample pred: [5 8 7 8 0 4 1 8 9 8 9 9 9 2 4 4 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 32.  36.  38.  16.  27.  29.  31.  26.  48.  30.]
Step 3100: val accuracy 0.5920
Step 3150: training accuracy 0.6080
    sample pred: [5 8 7 8 0 4 8 8 9 8 9 9 9 2 0 4 7 4 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 37.  39.  24.  23.  34.  12.  39.  24.  45.  27.]
Step 3150: val accuracy 0.6420
Step 3200: training accuracy 0.6480
    sample pred: [5 8 5 8 0 0 1 8 9 8 9 9 9 2 0 4 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 4600: training accuracy 0.6880
    sample pred: [5 8 7 8 2 4 1 8 9 8 9 9 9 2 3 4 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 27.  31.  32.  33.  34.  19.  39.  40.  51.  38.]
Step 4600: val accuracy 0.6440
Step 4650: training accuracy 0.7440
    sample pred: [5 8 7 8 0 0 1 8 9 8 0 9 9 2 7 4 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 41.  42.  32.  33.  37.  21.  36.  46.  50.  34.]
Step 4650: val accuracy 0.6980
Step 4700: training accuracy 0.7340
    sample pred: [5 8 7 8 0 0 1 8 9 8 3 9 9 2 7 4 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 43.  40.  34.  21.  35.  22.  43.  45.  45.  39.]
Step 4700: val accuracy 0.7100
Step 4750: training accuracy 0.7500
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 3 2 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 6150: training accuracy 0.8160
    sample pred: [5 8 7 8 2 4 1 8 9 8 2 9 9 2 7 5 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 41.  43.  48.  27.  42.  30.  44.  51.  42.  40.]
Step 6150: val accuracy 0.7160
Step 6200: training accuracy 0.8020
    sample pred: [7 8 9 8 0 4 1 8 9 8 2 9 9 2 9 4 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  45.  37.  36.  40.  15.  45.  48.  44.  42.]
Step 6200: val accuracy 0.7100
Step 6250: training accuracy 0.8400
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  45.  41.  36.  39.  25.  46.  51.  48.  39.]
Step 6250: val accuracy 0.7420
Step 6300: training accuracy 0.8360
    sample pred: [5 8 7 8 0 0 1 8 9 8 2 9 9 2 7 7 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 7700: training accuracy 0.8780
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 2 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  44.  49.  35.  46.  28.  48.  50.  48.  42.]
Step 7700: val accuracy 0.7500
Step 7750: training accuracy 0.8920
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 4 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  45.  46.  39.  45.  29.  47.  50.  53.  43.]
Step 7750: val accuracy 0.7440
Step 7800: training accuracy 0.8680
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 2 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 47.  45.  54.  36.  38.  33.  44.  49.  44.  44.]
Step 7800: val accuracy 0.7120
Step 7850: training accuracy 0.8800
    sample pred: [5 8 0 8 0 4 1 8 9 8 2 9 9 2 0 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

In [16]:
print(accuracy1_val)

[0.079999998, 0.086000003, 0.1, 0.093999997, 0.168, 0.2, 0.21799999, 0.226, 0.24600001, 0.30599999, 0.34, 0.31999999, 0.31, 0.35600001, 0.28400001, 0.34999999, 0.33199999, 0.34999999, 0.396, 0.40799999, 0.442, 0.43599999, 0.41800001, 0.458, 0.43399999, 0.454, 0.44999999, 0.456, 0.46599999, 0.426, 0.49200001, 0.5, 0.49399999, 0.528, 0.50400001, 0.50599998, 0.458, 0.53600001, 0.5, 0.55000001, 0.55599999, 0.56599998, 0.542, 0.53799999, 0.56599998, 0.58399999, 0.574, 0.574, 0.57599998, 0.58999997, 0.56800002, 0.58200002, 0.62199998, 0.61199999, 0.602, 0.60799998, 0.62400001, 0.61000001, 0.63800001, 0.616, 0.63, 0.62800002, 0.59200001, 0.64200002, 0.648, 0.65200001, 0.63599998, 0.58999997, 0.61799997, 0.62599999, 0.662, 0.65600002, 0.64200002, 0.648, 0.648, 0.69400001, 0.67199999, 0.662, 0.64999998, 0.65600002, 0.68400002, 0.64200002, 0.676, 0.65200001, 0.676, 0.64200002, 0.69199997, 0.68800002, 0.67799997, 0.68599999, 0.69400001, 0.65600002, 0.64399999, 0.69800001, 0.70999998, 0.69199997, 

In [17]:
print(accuracy1_train)

[0.090000004, 0.112, 0.097999997, 0.090000004, 0.198, 0.19599999, 0.20200001, 0.244, 0.236, 0.266, 0.30399999, 0.28999999, 0.296, 0.28600001, 0.27000001, 0.33000001, 0.31600001, 0.30399999, 0.35800001, 0.35800001, 0.38, 0.396, 0.40599999, 0.414, 0.40400001, 0.412, 0.42399999, 0.44800001, 0.46399999, 0.412, 0.47400001, 0.46000001, 0.46599999, 0.47999999, 0.48800001, 0.50400001, 0.472, 0.49599999, 0.50999999, 0.5, 0.53600001, 0.56599998, 0.546, 0.58200002, 0.58999997, 0.57200003, 0.57599998, 0.574, 0.59399998, 0.58200002, 0.57599998, 0.57200003, 0.616, 0.58600003, 0.59399998, 0.60399997, 0.61799997, 0.58200002, 0.61799997, 0.62599999, 0.662, 0.64600003, 0.62599999, 0.60799998, 0.648, 0.64200002, 0.64999998, 0.62800002, 0.64399999, 0.66000003, 0.67799997, 0.64399999, 0.67000002, 0.64999998, 0.68000001, 0.68000001, 0.67199999, 0.68000001, 0.69599998, 0.69800001, 0.68599999, 0.67799997, 0.708, 0.69199997, 0.69400001, 0.65600002, 0.73799998, 0.69, 0.704, 0.73000002, 0.74400002, 0.73000002, 0

In [19]:
accuracy2_val,accuracy2_train,time2=test_model('adam','vgg14',64,20001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.0900
    sample pred: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.   0.  45.   0.   0.   0.   0.]
Step   0: val accuracy 0.0800
Step  50: training accuracy 0.1040
    sample pred: [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.   0.   0.   0.  52.   0.   0.]
Step  50: val accuracy 0.1000
Step 100: training accuracy 0.1080
    sample pred: [8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.   0.   0.   0.   0.  54.   0.]
Step 100: val accuracy 0.0900
Step 150: training accuracy 0.1720
    sample pred: [6 1 1 8 1 1 1 1 1 1 1 1 1 4 1 4 1 6

Step 1500: training accuracy 0.4740
    sample pred: [5 8 5 8 0 9 1 8 1 8 4 9 9 4 6 4 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 28.  26.  18.   3.  20.  18.  38.  24.  45.  17.]
Step 1500: val accuracy 0.4860
Step 1550: training accuracy 0.4780
    sample pred: [5 8 5 0 0 1 1 0 1 8 7 9 0 7 6 5 7 5 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 33.  34.  10.  19.  10.  30.  27.  35.  33.   8.]
Step 1550: val accuracy 0.4600
Step 1600: training accuracy 0.4860
    sample pred: [5 8 5 8 0 1 9 9 1 8 7 9 9 4 6 4 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 32.  31.   4.  10.  35.  29.  21.  26.  34.  21.]
Step 1600: val accuracy 0.5020
Step 1650: training accuracy 0.5000
    sample pred: [5 8 5 8 0 9 1 8 9 8 9 9 9 5 6 6 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 3050: training accuracy 0.6380
    sample pred: [5 0 5 8 0 1 1 0 9 8 9 9 9 2 4 4 7 3 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 40.  37.  23.  25.  32.  17.  42.  37.  35.  31.]
Step 3050: val accuracy 0.6400
Step 3100: training accuracy 0.6020
    sample pred: [5 8 5 8 0 0 1 8 9 8 9 9 9 2 4 4 7 5 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 35.  40.  28.   4.  32.  28.  37.  22.  46.  29.]
Step 3100: val accuracy 0.6040
Step 3150: training accuracy 0.6620
    sample pred: [5 8 5 8 0 0 1 8 9 8 0 9 9 2 7 4 7 3 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 40.  39.  22.  31.  34.  17.  38.  33.  46.  31.]
Step 3150: val accuracy 0.6460
Step 3200: training accuracy 0.6280
    sample pred: [5 8 5 8 2 4 1 8 9 8 3 9 9 2 4 4 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 4600: training accuracy 0.7440
    sample pred: [5 8 7 8 0 4 1 8 9 8 0 9 9 2 7 2 7 2 8 3]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 42.  38.  40.  30.  39.  19.  42.  40.  47.  35.]
Step 4600: val accuracy 0.6880
Step 4650: training accuracy 0.7360
    sample pred: [5 8 7 8 0 4 1 8 9 8 9 9 9 2 7 4 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 35.  38.  37.  29.  32.  15.  45.  45.  51.  41.]
Step 4650: val accuracy 0.6880
Step 4700: training accuracy 0.6980
    sample pred: [5 8 7 8 0 1 1 8 9 8 3 9 9 2 7 3 7 3 8 7]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 40.  40.  30.  30.  26.  25.  29.  47.  42.  40.]
Step 4700: val accuracy 0.6820
Step 4750: training accuracy 0.7420
    sample pred: [5 1 7 8 2 4 1 8 9 8 3 9 9 2 2 4 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 6150: training accuracy 0.8080
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 5 7 2 8 3]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 42.  46.  40.  20.  36.  32.  48.  50.  51.  39.]
Step 6150: val accuracy 0.7100
Step 6200: training accuracy 0.7940
    sample pred: [5 8 2 8 2 0 1 8 9 8 0 9 9 2 7 2 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 44.  44.  51.  26.  23.  32.  45.  42.  47.  43.]
Step 6200: val accuracy 0.7120
Step 6250: training accuracy 0.7980
    sample pred: [5 8 7 8 0 4 1 8 9 8 0 9 9 2 7 5 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 45.  39.  34.  37.  36.  31.  40.  41.  52.  44.]
Step 6250: val accuracy 0.7360
Step 6300: training accuracy 0.7800
    sample pred: [5 8 7 8 2 4 1 8 9 8 2 9 9 2 7 4 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

...

    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  56.  42.  48.  44.  52.  51.  54.  47.]
Step 13800: val accuracy 0.7460
Step 13850: training accuracy 0.9700
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  47.  55.  41.  45.  44.  54.  48.  54.  47.]
Step 13850: val accuracy 0.7480
Step 13900: training accuracy 0.9740
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 4 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  55.  39.  49.  44.  53.  50.  53.  46.]
Step 13900: val accuracy 0.7560
Step 13950: training accuracy 0.9780
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  47.  56.  42.  48.  43

Step 15350: training accuracy 0.9880
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 3]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  47.  54.  45.  49.  45.  54.  51.  53.  47.]
Step 15350: val accuracy 0.7700
Step 15400: training accuracy 0.9460
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  45.  56.  41.  43.  39.  53.  49.  54.  44.]
Step 15400: val accuracy 0.7360
Step 15450: training accuracy 0.9440
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  46.  55.  42.  46.  44.  44.  49.  53.  43.]
Step 15450: val accuracy 0.7640
Step 15500: training accuracy 0.9660
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by

    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  52.  44.  49.  39.  53.  52.  54.  46.]
Step 16850: val accuracy 0.7460
Step 16900: training accuracy 0.9740
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  56.  43.  46.  43.  54.  48.  54.  45.]
Step 16900: val accuracy 0.7380
Step 16950: training accuracy 0.9860
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  46.  56.  45.  47.  45.  52.  52.  54.  46.]
Step 16950: val accuracy 0.7640
Step 17000: training accuracy 0.9880
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  46.  56.  45.  47.  45

Step 18400: training accuracy 0.9760
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  46.  53.  44.  48.  42.  52.  52.  53.  47.]
Step 18400: val accuracy 0.7580
Step 18450: training accuracy 0.9920
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  56.  45.  48.  44.  53.  51.  54.  47.]
Step 18450: val accuracy 0.7620
Step 18500: training accuracy 0.9840
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  46.  53.  45.  49.  43.  53.  52.  54.  47.]
Step 18500: val accuracy 0.7700
Step 18550: training accuracy 0.9720
    sample pred: [5 1 2 8 0 4 1 8 9 8 2 9 9 2 9 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by

    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  53.  45.  49.  44.  54.  52.  54.  46.]
Step 19900: val accuracy 0.7560
Step 19950: training accuracy 0.9880
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  46.  54.  44.  49.  43.  54.  52.  54.  47.]
Step 19950: val accuracy 0.7620
Step 20000: training accuracy 0.9880
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  55.  43.  49.  45.  53.  51.  54.  46.]
Step 20000: val accuracy 0.7440
[0.079999998, 0.1, 0.090000004, 0.19, 0.208, 0.24600001, 0.236, 0.222, 0.31400001, 0.242, 0.25, 0.28999999, 0.296, 0.31, 0.28600001, 0.28799999, 0.32800001, 0.32600001, 0.34999999, 0.352, 0.37599999, 0.384, 0.41, 0.449

In [20]:
print(accuracy2_val)

[0.079999998, 0.1, 0.090000004, 0.19, 0.208, 0.24600001, 0.236, 0.222, 0.31400001, 0.242, 0.25, 0.28999999, 0.296, 0.31, 0.28600001, 0.28799999, 0.32800001, 0.32600001, 0.34999999, 0.352, 0.37599999, 0.384, 0.41, 0.44999999, 0.40799999, 0.44999999, 0.458, 0.458, 0.45199999, 0.45199999, 0.486, 0.46000001, 0.50199997, 0.5, 0.49399999, 0.54000002, 0.484, 0.53200001, 0.546, 0.51999998, 0.53200001, 0.55000001, 0.51599997, 0.52399999, 0.53399998, 0.55800003, 0.56199998, 0.53799999, 0.588, 0.58999997, 0.57999998, 0.542, 0.55599999, 0.57200003, 0.59200001, 0.58200002, 0.602, 0.58399999, 0.602, 0.60399997, 0.60600001, 0.63999999, 0.60399997, 0.64600003, 0.64399999, 0.61400002, 0.65200001, 0.59399998, 0.65799999, 0.64200002, 0.62199998, 0.64399999, 0.64600003, 0.64399999, 0.65399998, 0.63599998, 0.63, 0.616, 0.65200001, 0.648, 0.65600002, 0.676, 0.65600002, 0.65799999, 0.66600001, 0.68599999, 0.65200001, 0.648, 0.69999999, 0.69199997, 0.69, 0.69, 0.68800002, 0.68800002, 0.68199998, 0.68000001, 0

In [ ]:
accuracy2_val,accuracy2_train,time2=test_model('adam','vgg16',64,20001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.1020
    sample pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
Step   0: val accuracy 0.1020
Step  50: training accuracy 0.1080
    sample pred: [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.   0.   0.  54.   0.   0.   0.]
Step  50: val accuracy 0.1060
Step 100: training accuracy 0.0940
    sample pred: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.  47.   0.   0.   0.   0.   0.   0.   0.   0.]
Step 100: val accuracy 0.1300
Step 150: training accuracy 0.1020
    sample pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Step 1500: training accuracy 0.3960
    sample pred: [5 8 3 8 7 0 9 9 9 8 6 9 7 4 4 6 7 5 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 27.  15.   1.   5.  30.  26.  29.  18.  28.  19.]
Step 1500: val accuracy 0.4180
Step 1550: training accuracy 0.4120
    sample pred: [5 8 3 0 5 1 1 9 1 8 7 9 0 7 6 3 7 7 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 34.  33.  14.  20.   1.  21.  16.  42.  17.   8.]
Step 1550: val accuracy 0.4100
Step 1600: training accuracy 0.4220
    sample pred: [5 8 3 8 7 0 8 9 9 8 7 9 7 4 6 6 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 22.  14.  11.   9.  19.  22.  30.  28.  36.  20.]
Step 1600: val accuracy 0.4480
Step 1650: training accuracy 0.4160
    sample pred: [5 8 5 8 7 1 9 9 1 8 6 9 9 4 6 6 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 3050: training accuracy 0.5660
    sample pred: [5 8 3 8 0 0 1 9 1 8 2 9 7 4 6 4 7 7 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 34.  33.  18.  27.  32.  13.  35.  37.  34.  20.]
Step 3050: val accuracy 0.5660
Step 3100: training accuracy 0.5560
    sample pred: [5 8 5 8 0 9 8 9 9 8 9 9 9 4 7 2 7 7 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 32.  28.  23.  10.  21.  23.  36.  36.  40.  29.]
Step 3100: val accuracy 0.5800
Step 3150: training accuracy 0.5000
    sample pred: [5 8 5 8 2 0 8 3 9 8 2 9 7 2 6 2 7 5 8 2]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 28.  29.  29.  12.  24.  20.  39.  23.  32.  14.]
Step 3150: val accuracy 0.5000
Step 3200: training accuracy 0.5560
    sample pred: [5 8 3 8 7 0 9 9 9 8 9 9 9 4 7 3 7 7 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 4600: training accuracy 0.6480
    sample pred: [5 8 7 8 8 0 9 8 9 8 8 9 9 4 4 3 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 35.  40.  20.  23.  38.  17.  39.  26.  49.  37.]
Step 4600: val accuracy 0.6300
Step 4650: training accuracy 0.6640
    sample pred: [5 8 5 8 0 0 9 8 9 8 8 9 9 2 7 3 7 3 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 38.  40.  25.  32.  37.  15.  33.  39.  46.  27.]
Step 4650: val accuracy 0.6240
Step 4700: training accuracy 0.6680
    sample pred: [5 8 5 8 8 0 9 8 9 8 8 9 9 2 7 2 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 41.  40.  29.  15.  32.  25.  38.  36.  49.  29.]
Step 4700: val accuracy 0.6280
Step 4750: training accuracy 0.6440
    sample pred: [5 8 5 8 2 4 8 8 9 8 8 9 9 2 4 2 7 5 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 6150: training accuracy 0.7640
    sample pred: [5 8 7 8 0 4 9 8 9 8 0 9 9 2 7 2 7 7 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 42.  40.  35.  28.  34.  21.  43.  46.  50.  43.]
Step 6150: val accuracy 0.6880
Step 6200: training accuracy 0.7260
    sample pred: [5 8 7 8 2 1 1 8 9 8 2 9 9 2 4 2 7 7 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 41.  43.  44.  27.  40.  17.  37.  31.  48.  35.]
Step 6200: val accuracy 0.6840
Step 6250: training accuracy 0.7700
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 3 7 5 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 44.  42.  36.  31.  30.  26.  43.  44.  50.  39.]
Step 6250: val accuracy 0.6920
Step 6300: training accuracy 0.7120
    sample pred: [5 8 7 8 0 1 1 8 9 8 8 9 9 2 7 3 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

# VGG16

In [5]:
accuracy2_val,accuracy2_train,time2=test_model('adam','vgg16',64,20001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.1020
    sample pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
Step   0: val accuracy 0.1020
Step  50: training accuracy 0.0900
    sample pred: [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.  45.   0.   0.   0.   0.   0.   0.]
Step  50: val accuracy 0.0940
Step 100: training accuracy 0.1040
    sample pred: [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [  0.   0.   0.   0.   0.   0.   0.  52.   0.   0.]
Step 100: val accuracy 0.1000
Step 150: training accuracy 0.1120
    sample pred: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

Step 1500: training accuracy 0.4940
    sample pred: [5 8 3 8 0 9 8 8 1 8 9 9 9 2 6 2 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 16.  19.  23.  23.  18.  16.  30.  33.  49.  20.]
Step 1500: val accuracy 0.4300
Step 1550: training accuracy 0.4540
    sample pred: [5 8 3 8 2 9 8 8 9 8 6 9 7 7 6 6 7 5 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 22.  14.  20.  22.   6.   8.  41.  35.  38.  21.]
Step 1550: val accuracy 0.4500
Step 1600: training accuracy 0.4000
    sample pred: [5 8 3 8 2 9 9 9 9 9 3 9 9 4 6 4 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 20.   0.  16.  35.  23.   4.  28.  10.  33.  31.]
Step 1600: val accuracy 0.3980
Step 1650: training accuracy 0.4300
    sample pred: [5 8 3 8 4 9 9 9 1 8 6 9 9 4 6 6 7 4 8 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 3050: training accuracy 0.5720
    sample pred: [5 8 5 8 0 0 8 8 9 8 8 9 9 2 3 2 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 36.  39.  31.  18.  20.  20.  25.  32.  48.  17.]
Step 3050: val accuracy 0.5400
Step 3100: training accuracy 0.6180
    sample pred: [5 8 5 8 0 9 1 8 9 8 9 9 9 2 6 2 7 7 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 37.  38.  35.  17.  25.  13.  33.  35.  43.  33.]
Step 3100: val accuracy 0.5980
Step 3150: training accuracy 0.6400
    sample pred: [5 8 5 8 0 0 1 8 9 8 8 9 9 2 6 2 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 39.  40.  31.  25.  25.  18.  37.  31.  47.  27.]
Step 3150: val accuracy 0.6140
Step 3200: training accuracy 0.5900
    sample pred: [5 8 5 8 0 0 8 8 9 8 3 9 7 2 6 2 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 4600: training accuracy 0.6960
    sample pred: [5 8 5 8 0 4 1 8 9 8 0 9 9 2 6 4 7 7 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 45.  39.  38.  14.  33.  27.  32.  40.  48.  32.]
Step 4600: val accuracy 0.6400
Step 4650: training accuracy 0.6720
    sample pred: [5 0 5 8 0 0 1 8 9 8 8 9 9 2 2 3 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 43.  41.  35.  19.  34.  24.  36.  28.  51.  25.]
Step 4650: val accuracy 0.6360
Step 4700: training accuracy 0.6800
    sample pred: [5 8 3 8 0 4 1 8 9 8 0 9 9 2 1 3 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 41.  41.  36.  32.  23.  11.  37.  37.  51.  31.]
Step 4700: val accuracy 0.6560
Step 4750: training accuracy 0.6740
    sample pred: [3 8 5 8 0 4 1 8 9 8 2 9 9 2 6 4 7 3 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

Step 6150: training accuracy 0.7460
    sample pred: [5 6 7 8 0 4 1 8 9 8 3 9 9 2 6 3 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 45.  43.  34.  26.  38.  23.  44.  39.  45.  36.]
Step 6150: val accuracy 0.7200
Step 6200: training accuracy 0.7500
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 4 3 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 44.  42.  39.  18.  40.  23.  46.  42.  46.  35.]
Step 6200: val accuracy 0.7100
Step 6250: training accuracy 0.7700
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 4 7 7 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 44.  40.  42.  16.  39.  17.  46.  48.  51.  42.]
Step 6250: val accuracy 0.7080
Step 6300: training accuracy 0.7340
    sample pred: [5 8 7 8 2 4 1 8 9 8 2 9 9 2 3 3 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class:

...

    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 3 5 7 0 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  47.  51.  41.  39.  39.  47.  49.  52.  44.]
Step 13800: val accuracy 0.7300
Step 13850: training accuracy 0.9260
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 4 3 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  47.  51.  34.  49.  33.  51.  51.  52.  46.]
Step 13850: val accuracy 0.7580
Step 13900: training accuracy 0.9280
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 3 5 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  43.  53.  37.  42.  40.  51.  50.  54.  45.]
Step 13900: val accuracy 0.7520
Step 13950: training accuracy 0.9340
    sample pred: [5 8 2 8 2 4 1 8 9 8 2 9 9 2 7 5 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 46.  47.  54.  38.  46.  41

Step 15350: training accuracy 0.9580
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 48.  47.  54.  41.  48.  40.  52.  52.  51.  46.]
Step 15350: val accuracy 0.7700
Step 15400: training accuracy 0.9440
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 2 7 5 8 3]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  44.  56.  41.  45.  40.  53.  50.  50.  43.]
Step 15400: val accuracy 0.7360
Step 15450: training accuracy 0.9620
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 2 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 48.  46.  55.  40.  48.  41.  52.  52.  53.  46.]
Step 15450: val accuracy 0.7740
Step 15500: training accuracy 0.9700
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by

    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  55.  42.  48.  40.  46.  51.  54.  45.]
Step 16850: val accuracy 0.7580
Step 16900: training accuracy 0.9360
    sample pred: [5 8 7 8 0 4 1 8 9 8 2 9 9 2 7 5 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  44.  46.  40.  47.  41.  51.  52.  52.  46.]
Step 16900: val accuracy 0.7600
Step 16950: training accuracy 0.9300
    sample pred: [5 8 2 8 0 4 1 2 9 8 2 9 9 2 7 5 7 5 8 3]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 46.  45.  55.  32.  48.  44.  51.  51.  49.  44.]
Step 16950: val accuracy 0.7540
Step 17000: training accuracy 0.9460
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 49.  44.  56.  41.  47.  42

Step 18400: training accuracy 0.9420
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 0 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  46.  56.  39.  41.  43.  49.  50.  51.  45.]
Step 18400: val accuracy 0.7260
Step 18450: training accuracy 0.9640
    sample pred: [5 8 2 8 0 4 1 8 9 8 5 9 9 2 7 5 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  47.  50.  40.  49.  43.  52.  51.  54.  46.]
Step 18450: val accuracy 0.7920
Step 18500: training accuracy 0.9680
    sample pred: [5 8 2 8 0 4 1 8 9 8 5 9 9 2 7 5 7 7 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  55.  42.  48.  43.  46.  52.  54.  46.]
Step 18500: val accuracy 0.7780
Step 18550: training accuracy 0.9420
    sample pred: [5 8 2 8 0 4 1 8 9 8 5 9 9 2 9 5 7 7 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by

    sample pred: [5 8 2 8 0 4 1 8 9 8 5 9 9 2 7 5 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  54.  40.  46.  45.  53.  51.  53.  46.]
Step 19900: val accuracy 0.7940
Step 19950: training accuracy 0.9700
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 9 5 7 2 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 50.  47.  56.  42.  46.  43.  52.  51.  54.  44.]
Step 19950: val accuracy 0.7780
Step 20000: training accuracy 0.9600
    sample pred: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 5 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 51.  47.  52.  43.  47.  42.  50.  52.  51.  45.]
Step 20000: val accuracy 0.7720
[0.102, 0.093999997, 0.1, 0.086000003, 0.168, 0.154, 0.23199999, 0.23999999, 0.19400001, 0.25, 0.30000001, 0.30199999, 0.33000001, 0.32600001, 0.33199999, 0.30000001, 0.34599999, 0.338, 0.34400001, 0.34, 0.40200001, 0.

In [6]:
print(accuracy2_val)

[0.102, 0.093999997, 0.1, 0.086000003, 0.168, 0.154, 0.23199999, 0.23999999, 0.19400001, 0.25, 0.30000001, 0.30199999, 0.33000001, 0.32600001, 0.33199999, 0.30000001, 0.34599999, 0.338, 0.34400001, 0.34, 0.40200001, 0.33399999, 0.36399999, 0.39399999, 0.36199999, 0.40599999, 0.41, 0.43799999, 0.47400001, 0.40799999, 0.43000001, 0.44999999, 0.398, 0.472, 0.41600001, 0.472, 0.46200001, 0.50599998, 0.51200002, 0.44999999, 0.52399999, 0.458, 0.542, 0.52200001, 0.55199999, 0.454, 0.546, 0.55400002, 0.53600001, 0.56999999, 0.52600002, 0.52399999, 0.58600003, 0.56800002, 0.56199998, 0.51999998, 0.57599998, 0.56199998, 0.55400002, 0.602, 0.60000002, 0.54000002, 0.59799999, 0.61400002, 0.55800003, 0.588, 0.56999999, 0.574, 0.59200001, 0.62199998, 0.59399998, 0.56400001, 0.63, 0.63, 0.616, 0.62, 0.63599998, 0.61199999, 0.63200003, 0.634, 0.63800001, 0.67400002, 0.62400001, 0.63999999, 0.65399998, 0.65200001, 0.676, 0.66600001, 0.62599999, 0.67000002, 0.66000003, 0.63200003, 0.63999999, 0.6359999